# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/binfo-234000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

# you sit West and hold:
hand = 'AKQJ95.J54..K953'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "2D", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': 'PASS', 'insta_score': 0.967}]

In [4]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '2S', 'insta_score': 0.226, 'expected_score': 93, 'adjustment': 11},
 {'call': 'PASS',
  'insta_score': 0.7234,
  'expected_score': 36,
  'adjustment': 36}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['x.KQT9xx.AKx.AQT Txx.Ax.QJ9xxx.Jx 8xx.8x.T8xx.8xxx AKQJ9x.Jxx..K9xx 0.79717 2D-P-P-X-P-P-P (7.6)  2D-P-P-X-P-P-2S-3H-P-P-P (8.3) ',
 '8xx.AQxx.Ax.A8xx xx.Txx.KQJT9x.Jx Tx.K98.8xxxx.QTx AKQJ9x.Jxx..K9xx 0.62044 2D-P-P-X-P-P-P (7.8)  2D-P-P-X-P-P-2S-P-P-P (8.7) ',
 'x.AKQ9x.A9xx.AJ8 T8x.8x.KQJTxx.Qx xxx.Txx.8xx.Txxx AKQJ9x.Jxx..K9xx 0.45153 2D-P-P-X-P-P-P (8.7)  2D-P-P-X-P-P-2S-3H-P-P-P (8.3) ',
 'T8x.KQxx.Kx.AQxx xx.AT8.AJTxxx.T8 xx.9xx.Q98xx.Jxx AKQJ9x.Jxx..K9xx 0.44652 2D-P-P-X-P-P-P (8.7)  2D-P-P-X-P-P-2S-P-P-P (10.3) ',
 'xx.AKQ9x.QJ.AQxx T8.T8.AKT8xx.8xx xxx.xxx.9xxxx.JT AKQJ9x.Jxx..K9xx 0.28965 2D-P-P-X-P-P-P (8.0)  2D-P-P-X-P-P-2S-3H-P-P-P (7.4) ',
 '8xx.AKQT.xx.AQTx Txx.8x.AK8xxx.Jx x.9xxx.QJT9x.8xx AKQJ9x.Jxx..K9xx 0.23851 2D-P-P-X-P-P-P (7.4)  2D-P-P-X-P-P-2S-P-P-P (9.6) ',
 '8xxx.AQT9.Ax.Axx Tx.Kx.KT9xxx.Q8x x.8xxx.QJ8xx.JTx AKQJ9x.Jxx..K9xx 0.19662 2D-P-P-X-P-P-P (7.3)  2D-P-P-X-P-P-2S-P-P-P (9.8) ',
 'T8x.QTxx.AJ.AQTx xxx.Kx.KQT8xx.8x x.A98x.9xxxx.Jxx AKQJ9x.Jxx..K9xx 0.1

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AKQJ9x.Jxx..K9xx Txx.AKx.KQx.T8xx 8.8xxx.ATxxx.AQx xxx.QT9.J98xx.Jx
AKQJ9x.Jxx..K9xx 8x.A98x.J8x.QJTx x.Txxx.AKTxx.A8x Txxx.KQ.Q9xxx.xx
AKQJ9x.Jxx..K9xx Txxx.KTx.QTxx.Qx .A98x.K98xx.AT8x 8xx.Qxx.AJxx.Jxx
AKQJ9x.Jxx..K9xx xxx.A8x.QJxxxx.J 8.QTxx.ATx.AQTxx Txx.K9x.K98x.8xx
AKQJ9x.Jxx..K9xx 8xxx.AKQx.JTxx.A .T9xx.AKxxx.QT8x Txx.8x.Q98x.Jxxx
AKQJ9x.Jxx..K9xx T8x.ATx.T8xxx.T8 x.K8xx.QJx.AQJxx xxx.Q9x.AK9xx.xx
AKQJ9x.Jxx..K9xx T8x.K8x.QT98x.Ax x.QTxx.AKxx.QJxx xxx.A9x.Jxxx.T8x
AKQJ9x.Jxx..K9xx 8xx.Tx.Axxxx.Q8x x.8xxx.KQ9x.ATxx Txx.AKQ9.JT8x.Jx
AKQJ9x.Jxx..K9xx 8xx.Ax.QJ9xx.Qxx x.Qxxx.AKxxx.JT8 Txx.KT98.T8x.Axx
AKQJ9x.Jxx..K9xx 8xx.AKx.Q9xx.Q8x x.98xx.A8xx.AJxx Txx.QTx.KJTxx.Tx
AKQJ9x.Jxx..K9xx 8xxx.Kxx.Axxx.Jx x.QT8x.KJT98.AQx Tx.A9x.Qxxx.T8xx
AKQJ9x.Jxx..K9xx 8xxx.AQxx.KQ9x.T x.KT8x.AJTx.QJxx Tx.9x.8xxxx.A8xx
AKQJ9x.Jxx..K9xx 8xx.Txx.AQ9x.Axx x.KQ8x.K8xx.QJTx Txx.A9x.JTxxx.8x
AKQJ9x.Jxx..K9xx xxx.AQTx.Q98x.Qx x.Kxxx.Axxxx.A8x T8x.98.KJTx.JTxx
AKQJ9x.Jxx..K9xx xxx.Q9.QJT9xx.T8 x.AK8x.Kxx.QJx